In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib

from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler,minmax_scale
scaler = StandardScaler()

# from sklearn.model_selection import GridSearchCV

In [2]:
# #PATH
path_info = "F:/178/Tugas Akhir/dataset/annotations/"
path_to_fitur = "F:/178/Tugas Akhir/dataset/coba fitur/"
path_to_experiment ="F:/178/Tugas Akhir/dataset/experiment/"
path_to_model = "F:/178/Tugas Akhir/dataset/model/"
path_to_predict = "F:/178/Tugas Akhir/dataset/predict/"

# path_info = "C:/Users/USER/Downloads/Tugas Akhir/dataset/annotations/"
# path_to_fitur = "C:/Users/USER/Downloads/Tugas Akhir/dataset/coba fitur/"
# path_to_experiment ="C:/Users/USER/Downloads/Tugas Akhir/dataset/experiment/"
# path_to_model = "C:/Users/USER/Downloads/Tugas Akhir/dataset/model/"
# path_to_predict = "C:/Users/USER/Downloads/Tugas Akhir/dataset/predict/"

In [3]:
def load(fitur):
    data = pd.read_csv(path_to_experiment + fitur+".csv",header=None)
    ground = pd.read_csv(path_info + "static_annotations.csv")

    arousal_mean = ground['mean_arousal']
    valence_mean = ground['mean_valence']
    
    return data,arousal_mean,valence_mean

def split(data,value):
    train_data, test_data, train_values, test_values = train_test_split(data,value,test_size=0.3,random_state=1000)
    
    return train_data, test_data, train_values, test_values
    
def predict(data,values,test,true,clf):
    trained = clf.fit(data,values)
    pred = clf.predict(test)
    df = pd.DataFrame(data=[pred,true])
    
    return pred,trained,df

def score(pred,true):
    r2 = r2_score(true, pred)
    
    return r2

# def gridsearch(data,true):
#     model = GPR()
# #     1.0 * RationalQuadratic(length_scale=1.0, alpha=0.1)
#     Cs = [1,1e1,1e2,1e3,1e4,1e5]
#     gammas = [1,1e-1,1e-2,1e-3,1e-4,1e-5]
#     parameters = {'C':Cs,'gamma':gammas}

#     clf = GridSearchCV(model, parameters, cv=10, scoring='r2')
#     clf.fit(data,true)

#     return clf.best_params_ , clf.best_score_

def regresion(clf,data,values):
    scores = cross_val_score(clf,data,values,scoring='r2',cv=10)
    
    return scores

def save_model(model,case,types):
    path = path_to_model+types+case+'norm_svr.sav' # save the model
    joblib.dump(model, path)

def save_predict(df1,df2,case):
    path = path_to_predict+case+"_norm_svr.xlsx" # save the model
    arousal = df1.T
    valence = df2.T
    merged = arousal.merge(valence, how='outer', left_index=True, right_index=True)
    merged.to_excel(path,header=None,index=None)

def norm(data):
    scaler.fit(data)
    normalized = scaler.transform(data)
    
    return normalized

# CASE 1

In [4]:
datafitur = "case1" #CASE 1

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

#split arousal
train_data_a, test_data_a, train_values_a, test_values_a = split(data,minmax_scale(arousal_mean.values,feature_range=(-1, 1)))
#split valence
train_data_v, test_data_v, train_values_v, test_values_v = split(data,minmax_scale(valence_mean.values,feature_range=(-1, 1))) 

(744, 26)


GRIDSEARCHCV

In [5]:
# param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
# param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

In [6]:
# print("parameter Arousal:\t",param_a)
# print("parameter Valence:\t",param_v)

INITIAL MODEL

In [7]:
kernel = RationalQuadratic()

svr_case1_a = GPR(kernel) #init Arousal
svr_case1_v = GPR(kernel) #init Valence

TRAIN

In [10]:
# #Predict Arousal
a_a,train_a,df1_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, svr_case1_a)
# #Predict Valence
a_v,train_v,df1_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, svr_case1_v)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

AROUSAL

In [12]:
df1_a = df1_a.round(2)

display(df1_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,-0.04,0.16,0.39,-0.58,-0.42,-0.24,-0.48,0.21,-0.49,0.08,...,0.34,0.0,-0.04,-0.06,0.01,-0.03,0.18,0.33,-0.3,-0.64
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.0,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.5,-0.82


R2 Score Arousal:	0.55


VALENCE

In [13]:
df1_v = df1_v.round(2)

display(df1_v.head())
print("R2 Score Valence: %0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.17,0.17,0.09,-0.18,-0.17,-0.04,-0.14,0.12,-0.25,0.17,...,0.03,0.10,0.16,0.11,0.22,0.11,-0.00,0.16,-0.05,-0.29
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence: 0.04


REGRESSION

In [11]:
scores_a = regresion(svr_case1_a,data,arousal_mean) #Arousal Regression
scores_v = regresion(svr_case1_v,data,valence_mean) #Arousal Regression

AROUSAL

In [12]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.30138761  0.43375938  0.45214514  0.40027266  0.56706159  0.45007487
  0.53312179  0.46030968  0.59691538  0.51537081]
R2 Score Valence:	0.47 (+/- 0.08)


VALENCE

In [13]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [-0.31196621 -0.10182823  0.05031216 -0.22670515  0.21483656  0.07997565
  0.11794779  0.17116535  0.1961936  -0.02303394]
R2 Score Valence:	0.02 (+/- 0.17)


In [14]:
# save_predict(df1_a.round(2),df1_v.round(2),datafitur)
# save_model(train_a,datafitur,'arousal/') #save model Arousal
# save_model(train_v,datafitur,'valence/') #save model Valence

# CASE 2

In [20]:
datafitur = "case2" #CASE 2

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,minmax_scale(arousal_mean.values,feature_range=(-1, 1))) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,minmax_scale(valence_mean.values,feature_range=(-1, 1))) #split valence

(744, 34)


GRIDSEARCH

In [16]:
# param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
# param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

In [17]:
# print("parameter Arousal:\t",param_a)
# print("parameter Valence:\t",param_v)

INITIAL MODEL

In [21]:
kernel = RationalQuadratic()

svr_case2_a = GPR(kernel) #init Arousal
svr_case2_v = GPR(kernel) #init Valence

TRAIN

In [22]:
# #Predict Arousal
a_a,train_a,df2_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, svr_case2_a)
# #Predict Valence
a_v,train_v,df2_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, svr_case2_v)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

AROUSAL

In [23]:
df2_a = df2_a.round(2)

display(df2_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.07,0.11,0.42,-0.40,-0.33,-0.18,-0.50,0.26,-0.50,-0.01,...,0.43,0.03,-0.10,-0.07,0.09,-0.19,0.23,0.33,-0.33,-0.68
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.00,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.50,-0.82


R2 Score Arousal:	0.60


VALENCE

In [24]:
df2_v = df2_v.round(2)

display(df2_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.16,0.18,0.16,-0.12,-0.10,0.02,-0.22,0.17,-0.22,0.12,...,0.23,0.05,0.11,0.12,0.13,-0.00,0.16,0.24,-0.19,-0.20
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence:	0.17


REGRESSION

In [22]:
scores_a = regresion(svr_case2_a,data,arousal_mean) #Arousal Regression
scores_v = regresion(svr_case2_v,data,valence_mean) #Arousal Regression

AROUSAL

In [23]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.46623269  0.42964468  0.41222512  0.36998899  0.61357131  0.47704016
  0.43812026  0.49579152  0.54392657  0.52305608]
R2 Score Valence:	0.48 (+/- 0.07)


VALENCE

In [24]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.00871648  0.08598874 -0.01422386 -0.28123633  0.31157909  0.22491126
  0.14138847  0.16972624  0.11803513 -0.01656214]
R2 Score Valence:	0.07 (+/- 0.16)


In [25]:
# save_predict(df2_a.round(2),df2_v.round(2),datafitur)
# save_model(train_a,datafitur,'arousal/') #save model Arousal
# save_model(train_v,datafitur,'valence/') #save model Valence

# CASE 3

In [25]:
datafitur =  "case3" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,arousal_mean) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,valence_mean) #split valence

(744, 37)


GRIDSEARCH

In [27]:
# param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
# param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

In [28]:
# print("parameter Arousal:\t",param_a)
# print("parameter Valence:\t",param_v)

INITAL MODEL

In [26]:
kernel = RationalQuadratic()

svr_case3_a = GPR(kernel) #init Arousal
svr_case3_v = GPR(kernel) #init Valence

TRAIN

In [27]:
# #Predict Arousal
a_a,train_a,df3_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, svr_case3_a)
# #Predict Valence
a_v,train_v,df3_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, svr_case3_v)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

AROUSAL

In [28]:
df3_a = df3_a.round(2)

display(df3_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,5.13,5.47,6.29,4.0,3.67,4.43,3.27,5.91,3.53,4.96,...,6.23,5.22,4.51,4.71,5.3,4.32,5.81,6.15,4.01,3.31
1,6.20,5.90,6.40,3.1,3.50,3.60,3.00,5.80,3.60,6.20,...,7.00,5.00,3.70,5.80,4.7,4.90,4.80,6.30,3.30,2.20


R2 Score Arousal:	0.61


VALENCE

In [29]:
df3_v = df3_v.round(2)

display(df3_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,5.18,5.41,5.29,4.48,4.33,4.88,4.15,5.55,4.08,5.31,...,5.41,5.26,5.22,5.31,5.38,4.88,5.38,5.56,4.39,4.03
1,2.70,6.40,6.00,2.70,5.00,3.20,4.60,4.30,4.10,6.70,...,6.60,4.80,2.50,5.60,3.80,4.40,6.50,6.50,4.00,3.50


R2 Score Valence:	0.18


REGRESSION

In [33]:
scores_a = regresion(svr_case3_a,data,arousal_mean) #Arousal Regression
scores_v = regresion(svr_case3_v,data,valence_mean) #Arousal Regression

AROUSAL

In [34]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.46620522  0.42977377  0.41219533  0.36999433  0.61359349  0.47733101
  0.4380642   0.49585552  0.54391422  0.52308697]
R2 Score Valence:	0.48 (+/- 0.07)


VALENCE

In [35]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.00871914  0.08597777 -0.01417556 -0.28126208  0.31154689  0.22490856
  0.14141153  0.16973627  0.11807917 -0.01656913]
R2 Score Valence:	0.07 (+/- 0.16)


In [36]:
# save_predict(df3_a.round(2),df3_v.round(2),datafitur)
# save_model(train_a,datafitur,'arousal/') #save model Arousal
# save_model(train_v,datafitur,'valence/') #save model Valence

# CASE 4

In [30]:
datafitur =  "case4" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,arousal_mean) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,valence_mean) #split valence

(744, 61)


GRIDSEAERCH

In [38]:
# param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
# param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

In [39]:
# print("parameter Arousal:\t",param_a)
# print("parameter Valence:\t",param_v)

INITIAL MODEL

In [31]:
kernel = RationalQuadratic()

svr_case4_a = GPR(kernel) #init Arousal
svr_case4_v = GPR(kernel) #init Valence

TRAIN

In [32]:
# #Predict Arousal
a_a,train_a,df4_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, svr_case4_a)
# #Predict Valence
a_v,train_v,df4_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, svr_case4_v)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

AROUSAL

In [33]:
df4_a = df4_a.round(2)

display(df4_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,5.34,5.15,6.57,3.95,3.73,4.19,3.35,5.99,3.5,4.85,...,6.44,5.12,4.58,4.87,5.4,4.49,5.77,6.56,4.0,3.35
1,6.20,5.90,6.40,3.10,3.50,3.60,3.00,5.80,3.6,6.20,...,7.00,5.00,3.70,5.80,4.7,4.90,4.80,6.30,3.3,2.20


R2 Score Arousal:	0.61


VALENCE

In [34]:
df4_v = df4_v.round(2)

display(df4_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,5.18,5.15,5.53,4.39,4.47,4.82,4.17,5.46,4.01,5.14,...,5.49,4.96,5.23,5.24,5.35,4.63,5.29,5.72,4.37,3.98
1,2.70,6.40,6.00,2.70,5.00,3.20,4.60,4.30,4.10,6.70,...,6.60,4.80,2.50,5.60,3.80,4.40,6.50,6.50,4.00,3.50


R2 Score Valence:	0.20


REGRESSION

In [44]:
scores_a = regresion(svr_case4_a,data,arousal_mean) #Arousal Regression
scores_v = regresion(svr_case4_v,data,valence_mean) #Arousal Regression

AROUSAL

In [45]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.46618596  0.42976925  0.41232317  0.36997523  0.61358414  0.47729353
  0.43815196  0.49583003  0.54391906  0.52310651]
R2 Score Valence:	0.48 (+/- 0.07)


VALENCE

In [46]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.00872724  0.08597239 -0.01415968 -0.28125173  0.31154466  0.22489161
  0.14142607  0.16973975  0.11809055 -0.01655985]
R2 Score Valence:	0.07 (+/- 0.16)


In [47]:
# save_predict(df4_a.round(2),df4_v.round(2),datafitur)
# save_model(train_a,datafitur,'arousal/') #save model Arousal
# save_model(train_v,datafitur,'valence/') #save model Valence